In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt

batchsize = 128
epochs=25
loginterval=10

SEED = 1
torch.manual_seed(SEED)
cuda = torch.cuda.is_available()

#device = torch.device("cuda" if cuda else "cpu")
device = torch.device("cpu")



kwargs = {'num_workers': 4, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batchsize, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batchsize, shuffle=True, **kwargs)

In [2]:
class vae(nn.Module):
    def __init__(self):
        
        super(vae, self).__init__()
        # encoding_layers :FC (784 -> 400), 2 FC (mu and var) (400 -> 20)
        self.fc1=nn.Linear(784,400)
        self.fcmu=nn.Linear(400,20)
        self.fcvar=nn.Linear(400,20)
        
        # decoder :FC (20 -> 400), RELU activation; FC (400, 784)
        self.decoder = nn.Sequential(
                nn.Linear(20, 400),
                nn.ReLU(),
                nn.Linear(400, 784),
                )

    def encode(self, x):
        temp=F.relu(self.fc1(x))
        return self.fcmu(temp) , self.fcvar(temp)
    
    def reparametrize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        
        return eps.mul(std).add_(mu)

    def decode(self, z):
        return torch.sigmoid(self.decoder(z))

    def forward(self, x):
        xview = x.view(-1,784)
        mu, logvar = self.encode(xview)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar

mod = vae().to(device)
optimizer = optim.Adam(mod.parameters(), lr=1e-3)

In [3]:
def loss_function(recon_batch, x, mu, logvar, beta):
    # compute binary cross entropy summed across the batch
    reconstruction_function = nn.MSELoss(size_average=False)
    BCE = reconstruction_function(recon_batch, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)

    # the beta factor allows disentanglement for values >>1
    return BCE+KLD*beta

writing_path="../generated_vae/mod1/"
losses=list()

In [6]:
def train(epoch,model,l_function):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data.view(-1, 784), mu, logvar, beta)
        losses.append(loss.to(device).item())
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % loginterval == 0:
            print('\r Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.1f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)), end='')

    print(' Average loss: {:.1f}'.format(
          train_loss / len(train_loader.dataset)), end='')

def test(epoch,writing_path,model,l_function):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data.view(-1, 784), mu, logvar,beta)

            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batchsize, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         writing_path + 'reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print(' Test loss: {:.1f}'.format(test_loss))

In [7]:
beta=20
for epoch in range(1, epochs + 1):
    train(epoch,mod,loss_function)
    test(epoch,writing_path,mod,loss_function)
    with torch.no_grad():
     sample = torch.randn(64, 20).to(device)
     sample = mod.decode(sample).cpu()
     save_image(sample.view(64, 1, 28, 28), writing_path + 'sample_' + str(epoch) + '.png')

 Train Epoch: 1 [58880/60000 (98%)]	Loss: 53.0 Average loss: 55.9 Test loss: 53.4
 Train Epoch: 2 [58880/60000 (98%)]	Loss: 51.4 Average loss: 53.1 Test loss: 53.2
 Train Epoch: 3 [58880/60000 (98%)]	Loss: 51.2 Average loss: 53.0 Test loss: 53.2
 Train Epoch: 4 [58880/60000 (98%)]	Loss: 51.7 Average loss: 53.0 Test loss: 53.1
 Train Epoch: 5 [58880/60000 (98%)]	Loss: 51.7 Average loss: 52.9 Test loss: 53.1
 Train Epoch: 6 [58880/60000 (98%)]	Loss: 53.3 Average loss: 52.9 Test loss: 53.1
 Train Epoch: 7 [58880/60000 (98%)]	Loss: 51.8 Average loss: 52.9 Test loss: 53.0
 Train Epoch: 8 [58880/60000 (98%)]	Loss: 53.7 Average loss: 52.9 Test loss: 53.0
 Train Epoch: 9 [58880/60000 (98%)]	Loss: 52.0 Average loss: 52.9 Test loss: 53.0
 Train Epoch: 10 [58880/60000 (98%)]	Loss: 54.4 Average loss: 52.8 Test loss: 53.0
 Train Epoch: 11 [58880/60000 (98%)]	Loss: 55.1 Average loss: 52.8 Test loss: 53.0
 Train Epoch: 12 [58880/60000 (98%)]	Loss: 51.7 Average loss: 52.8 Test loss: 52.9
 Train Epoch:

In [ ]:
plt.plot(losses)